# Домашнее задание к лекции «Понятие класса»

## Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [24]:
import requests

def get_max_valute_value():
    r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()
    max_val = 0.0
    result = ''
    for valute in r['Valute']:
        if r['Valute'][valute]['Value'] > max_val:
            max_val = r['Valute'][valute]['Value']
            result = r['Valute'][valute]['Name']
    return result

In [25]:
get_max_valute_value()

'СДР (специальные права заимствования)'

## Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [96]:
from libs.exchange import Rate

class RateChanges(Rate):
    def __init__(self):
        super().__init__()
        # чтобы сравнивать с предыдущими показаниями, нужно получить список значений валют в текущий момент времени
        response = self.exchange_rates()
        self.xr = {k:v['Value'] for (k, v) in response.items()}
    
    # переопределяем метод make_format, чтобы учесть условия задания
    def make_format(self, currency, diff):
        """
        Возвращает текущее значение валюты при diff=False или 
        разницу с предыдущим значением при diff=True
        """
        response = self.exchange_rates()
        
        return_value = 0.0
        if currency in response:
            if diff:
                # считаем разницу между новым и прошлым значением
                return_value =  response[currency]['Value'] - self.xr[currency]
            else:
                # просто возвращаем новое значение
                return_value =  response[currency]['Value']
            # обновляем словарь значений валют новым значением
            self.xr[currency] = response[currency]['Value']
            return return_value
        else:
            return 'Error'
    
    def eur(self, diff=False):
        """Возвращает курс евро на сегодня при diff=False либо разницу с предыдущим значением при diff=True"""
        return self.make_format('EUR', diff)
    
    def usd(self, diff=False):
        """Возвращает курс доллара на сегодня при diff=False либо разницу с предыдущим значением при diff=True"""
        return self.make_format('USD', diff)
    
    def brl(self, diff=False):
        """Возвращает курс бразильского реала на сегодня при diff=False либо разницу с предыдущим значением при diff=True"""
        return self.make_format('BRL', diff)

In [97]:
rc = RateChanges()
rc.eur(True)

0.0

## Задание 3
Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем.

In [104]:
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.seniority, self.grade)
    
    def should_upgrade(self):
        pass
    
class Designer(Employee):
    def __init__(self, name):
        super().__init__(name, seniority=2)
    
    def should_upgrade(self):
        # для дизайнеров количество премий не меняется со стажем
        # поэтому изменение грейда может произойти только при получении премии
        # self.seniority += 1
        
        # условие повышения сотрудника 
        if self.seniority % 7 == 0:
            self.grade_up()
    
    def add_award(self, international=False):
        self.seniority += 1
        self.should_upgrade()
        if international:
            self.seniority += 1
            self.should_upgrade()
            
        # публикация результатов
        return self.publish_grade()

In [112]:
oleg = Designer('Олег')

In [113]:
import random
for i in range(20):
    # назначем Олегу премию случайным образом (обычную или международную)
    oleg.add_award(bool(random.getrandbits(1)))

Олег 4 1
Олег 5 1
Олег 7 2
Олег 9 2
Олег 11 2
Олег 12 2
Олег 13 2
Олег 15 3
Олег 17 3
Олег 19 3
Олег 21 4
Олег 23 4
Олег 25 4
Олег 26 4
Олег 27 4
Олег 28 5
Олег 30 5
Олег 31 5
Олег 32 5
Олег 34 5
